In [1]:
import os
import torch
import torchvision
import numpy as np
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
from skimage import io, transform
from tensorflow.examples.tutorials.mnist import input_data


In [2]:
# define some parameter
if not os.path.exists('./results'):
    os.mkdir('./results')


def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 3, 360, 270)
    return x
num_epochs = 200
batch_size = 128
learning_rate = 1e-3

#transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])
img_transform = transforms.Compose([
    transforms.ToTensor(),
    ])


In [4]:
# load image
mnist = input_data.read_data_sets('../../MNIST', one_hot=True)
z_dim = 10
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
h_dim = 128
cnt = 0

print X_dim

Extracting ../../MNIST/train-images-idx3-ubyte.gz
Extracting ../../MNIST/train-labels-idx1-ubyte.gz
Extracting ../../MNIST/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST/t10k-labels-idx1-ubyte.gz
784


In [ ]:
# load auto encoder
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(X_dim, h_dim),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(h_dim, X_dim), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
# instantiate model and define optimizer
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)